# Laboratorio 1 - Minería de Datos

1. Exploración rápida de sus datos

2. Tipo de cada una de las variables 

3. Investigue si las variables cuantitativas siguen una distribución normal y haga una
tabla de frecuencias

4. Preguntas

4.1.  ¿Cuáles son las 10 películas que contaron con más presupuesto?

4.2. ¿Cuáles son las 10 películas que más ingresos tuvieron?

4.3. ¿Cuál es la película que más votos tuvo?